## 1. System Initialization

Live feedback system for defect classification with dynamic cluster adaptation

In [ ]:
from matplotlib import gridspec
import matplotlib.pyplot as plt
import numpy as np
import pickle
from pathlib import Path
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix, classification_report


import skimage.io
from scipy.spatial.distance import cdist

import sys
sys.path.insert(0,'../')
from helper import classification_tools as ct
from helper import visualize as vis

%matplotlib inline

In [ ]:
# fc1 features saved from the previous step
fc1_path = Path('..','data','features','VGG16_fc1_features_std.pickle')
assert fc1_path.is_file()

# label encoder model which converts string labels to integers.
le_path = Path('..','models','label_encoder.pickle')
assert le_path.is_file()

# load the data and label encoder into memory
with open(fc1_path, 'rb') as f:
    data = pickle.load(f)

with open(le_path, 'rb') as f:
    le = pickle.load(f)

In [ ]:
files = data['filename']  # file paths to each image
fc1 = data['features']  # array containing fc1 features for each file
labels = data['labels']  # string labels for each image
y_gt = le.transform(labels)  # integer labels for each image

## 2. Feedback System Implementation


In [1]:
import pickle
import numpy as np
from pathlib import Path
import tensorflow.keras as keras

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from predictive_model import le # Import label encoder

class FeedbackSystem:
    def __init__(self):
        self.model = None
        self.pca = None
        self.new_data = []
        self.new_labels = []
        self.n_clusters = 7  # Initial cluster count
        self.load_initial_model()

    def load_initial_model(self):
        """Load pre-trained models with fixed PCA components"""
        self.pca = PCA(n_components=50, whiten=True, svd_solver='full')
        self.pca.fit(fc1)  # Fit on initial dataset
        
        # Initial clustering
        self.model = KMeans(n_clusters=self.n_clusters, init='k-means++', n_init=10)
        self.model.fit(self.pca.transform(fc1))

    def process_feedback(self, features):
        """Process user feedback for a single image"""
        reduced = self.pca.transform([features])
        cluster = self.model.predict(reduced)[0]
        predicted_label = le.inverse_transform([cluster])[0]

        is_correct = input(f"Predicted: {predicted_label}. Correct? (y/n): ").lower() == 'y'
        
        if not is_correct:
            is_new_class = input("New class? (y/n): ").lower() == 'y'
            if is_new_class:
                new_class = input("Class name: ").strip()
                if new_class not in le.classes_:
                    le.classes_ = np.append(le.classes_, new_class)
                    self.n_clusters += 1
                    print(f"Added '{new_class}'. Total clusters: {self.n_clusters}")
                else:
                    print("Class exists. Using existing label.")
            else:
                new_class = input("Correct label: ").strip()
                if new_class not in le.classes_:
                    print("Error: Invalid label")
                    return

            self.new_data.append(features)
            self.new_labels.append(new_class)
            print("Feedback stored. Retrain when ready.")
        else:
            print("Correct prediction recorded.")

    def retrain_model(self):
        """Quick retrain using only new feedback data"""
        if not self.new_data:
            print("No new data to retrain")
            return

        # Convert labels to integers
        encoded_labels = le.transform(self.new_labels)
        
        # Apply PCA transform
        transformed = self.pca.transform(self.new_data)
        
        # Retrain with updated cluster count
        self.model = KMeans(
            n_clusters=self.n_clusters,
            init='k-means++',
            n_init=10  # Reduced for faster training
        )
        self.model.fit(transformed)
        
        print(f"Model retrained with {self.n_clusters} clusters")

## 3. Feature Extraction Functions

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image

def extract_features(img_path):
    """Feature extractor for live images"""
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)
    return img_array.flatten()  # Simplified for demo

## 4. Demonstration Workflow

In [ ]:
# Initialize feedback system
fb_system = FeedbackSystem()

In [ ]:
# Demo sequence
demo_images = [
    ("new_class_1.bmp", True),
    ("new_class_2.bmp", True),
    ("existing_class.bmp", False)
]

for img_path, is_new in demo_images:
    features = extract_features(img_path)
    fb_system.process_feedback(features)
    
    if is_new:
        fb_system.retrain_model()

## 5. Visualization (Optional)

Optional: Add visualization code from 03_standard.ipynb here

Include t-SNE plots or confusion matrices if time permits